https://ambermd.org/tutorials/basic/tutorial1/section2.htm

In [1]:
%%file nuc.nab
molecule m;
m = fd_helix( "abdna", "aaaaa", "dna" );
putpdb( "1crn.pdb", m, "-wwpdb"); 

Writing nuc.nab


In [2]:
!/home/pi/amber20/bin/nab nuc.nab

./nuc crée un fichier nuc.pdb

In [3]:
import py3Dmol;import pubchempy as pcp
bromomethane=pcp.get_compounds(
'GLY', 'name')[0]
x=py3Dmol.view(query='cid:{}'.format(
bromomethane.cid), width=300,
height=300).setStyle({'stick': {
'radius': .05},'sphere': {'scale': 0.25}});x

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
import warnings;warnings.filterwarnings('ignore')
from __future__ import print_function
import pytraj as pt;import nglview as nv
traj = pt.load('1crn.pdb')
view = nv.show_pytraj(traj) 
view

NGLWidget()

In [5]:
traj = pt.load('polyAT_vac_init_min.pdb')
view = nv.show_pytraj(traj) 
view

RuntimeError: n_atoms = 0: make sure to load correct Topology filename or load supported topology (pdb, amber parm, psf, ...)

In [ ]:
with open("polyAT_vac_init_min.pdb") as ifile:
      system = "".join([x for x in ifile])
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
#view.setStyle({'stick': {'radius': .05},'sphere': {'scale': 0.25}})
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo();view.show()

In [ ]:
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
#view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.setStyle({'stick': {'radius': .05},'sphere': {'scale': 0.25}})
view.zoomTo();view.show()

In [ ]:
with open("1crn.pdb") as ifile:
    view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
#view.setStyle({'stick': {'radius': .05},'sphere': {'scale': 0.25}})
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo();view.show()

In [ ]:

i = 0
for line in system.split("\n"):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue
    if split[3] == "GLY":
        color = "yellow"
    else:
        color = "spectrum"
    idx = int(split[1])
    view.setStyle({'stick': {'radius': .05},'sphere': {'scale': 0.25}})
    #view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})
    i += 1
view.zoomTo()
view.show()


In [ ]:
ion = "ATOM   1  NE   NE    1    10.047  14.099   3.625  0.1812  1.5500     NE"
system2 = system + ion

view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system2)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})

i = 0
for line in system2.split("\n"):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue
    if split[-1] == "NE":
        view.setStyle({'stick': {'radius': .05},'sphere': {'scale': 0.25}})
        #view.setStyle({'model': -1, 'serial': i+1}, {"sphere": {'color': "pink"}})
    i += 1

view.zoomTo()
view.show()


In [ ]:
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(int(line[22:26]))
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip()

    def __str__(self):
        line = list(" " * 80)
        
        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].ljust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].ljust(3)
        line[22:26] = str(self["resid"]).ljust(4)
        line[30:38] = str(self["x"]).rjust(8)
        line[38:46] = str(self["y"]).rjust(8)
        line[46:54] = str(self["z"]).rjust(8)
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if "ATOM" in line or "HETATM" in line:
                self.append(Atom(line))
            
    def __str__(self):
        outstr = ""
        for at in self:
            outstr += str(at)
            
        return outstr

In [ ]:
with open("nuc.pdb") as ifile:
    mol = Molecule(ifile)

In [ ]:

for at in mol:
    if at["resname"] == "PRO":
        at["pymol"] = ({'stick': {'radius': .05},
                        'sphere': {'scale': 0.25,'color': "pink"}})
    elif at["resname"] == "GLY":
        at["pymol"] = ({'stick': {'radius': .05},
                        'sphere': {'scale': 0.25,'color': "pink"}})
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(str(mol))
for i, at in enumerate(mol):
    default = ({'stick': {'radius': .05},'sphere': {'scale': 0.25}})
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
view.zoomTo();view.show()